<a href="https://colab.research.google.com/github/DevanMayer/EE-480-Assignments/blob/main/ca5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **EE 480 - Coding Assignment #5: The Newton-Raphson Power Flow**

**Instructor:** Dr. Saeed Manshadi

**Due Date:** Oct 14, 2025

### **Instructions**

This assignment is a deep dive into the Newton-Raphson (NR) power flow method. You will implement the NR algorithm from scratch for small systems and then use the powerful `PowerModels.jl` package to analyze and solve a problem on a larger test case.

**Working in Google Colab:**
*   This notebook is designed to be completed in Google Colab.
*   **CRITICAL FIRST STEP:** Ensure the Julia kernel is active. If needed, run the setup cell from a previous assignment and restart the runtime.

**Submission:**
*   Please follow the submission instructions provided on Canvas to save your completed notebook back to your private GitHub repository.

---
### **Required Packages (Run This Cell First!)**

In [ ]:
# This cell adds the necessary packages for this assignment.
using Pkg
Pkg.add("PowerModels")
Pkg.add("Ipopt")
Pkg.add("LinearAlgebra")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ─────────────── v0.8.5
   Installed Distances ──────────────── v0.10.12
   Installed NLsolve ────────────────── v4.5.1
   Installed FiniteDiff ─────────────── v2.28.1
   Installed BenchmarkTools ─────────── v1.6.0
   Installed Memento ────────────────── v1.4.1
   Installed MutableArithmetics ─────── v1.6.6
   Installed MathOptInterface ───────── v1.45.0
   Installed NLSolversBase ──────────── v7.10.0
   Installed StructTypes ────────────── v1.11.0
   Installed JSON3 ──────────────────── v1.14.3
   Installed LineSearches ───────────── v7.4.0
   Installed InfrastructureModels ───── v0.7.8
   Installed UnPack ─────────────────── v1.0.2
   Installed PowerModels ────────────── v0.21.5
   Installed Parameters ─────────────── v0.12.3
   Installed JuMP ───────────────────── v1.29.1
   Installed DifferentiationInterface ─ v0.7.9
    Updating `~/.julia/environments/v1.11/Project.t

---
### **Problem 1**

A generator bus (with a 1.0 per unit voltage) supplies a 180 MW, 60 Mvar load through a lossless transmission line with per unit (100 MVA base) impedance of `j0.1` and no line charging. This is a simple two-bus system.
*   **Bus 1:** Generator (Slack Bus), `V₁ = 1.0∠0°` p.u.
*   **Bus 2:** Load Bus, `S_L = 1.8 + j0.6` p.u.

Starting with an initial voltage guess of `V₂ = 1.0∠0°`, iterate until converged using the Newton-Raphson power flow method. For convergence criteria, use a maximum power flow mismatch (`ΔP` or `ΔQ`) of 0.001 p.u. (equivalent to 0.1 MVA on a 100 MVA base).

**Guidance:**
*   The unknowns are `δ₂` and `|V₂|`. Your Jacobian will be a 2x2 matrix.
*   **Step 1:** Build the 2x2 Ybus matrix for this system.
*   **Step 2:** Start a `for` loop for your iterations.
*   **Step 3 (Inside Loop):** Calculate the power injections `P₂_calc` and `Q₂_calc` based on the current voltage estimates.
*   **Step 4 (Inside Loop):** Calculate the mismatches `ΔP₂` and `ΔQ₂`. Check if `max(abs(ΔP₂), abs(ΔQ₂))` is less than your tolerance. If it is, `break` the loop.
*   **Step 5 (Inside Loop):** Calculate the four elements of the 2x2 Jacobian matrix (`∂P₂/∂δ₂`, `∂P₂/∂|V₂|`, etc.).
*   **Step 6 (Inside Loop):** Solve for the updates `[Δδ₂; Δ|V₂|] = J⁻¹ * [ΔP₂; ΔQ₂]`.
*   **Step 7 (Inside Loop):** Update your voltage angle and magnitude: `δ₂ = δ₂ + Δδ₂` and `|V₂| = |V₂| + Δ|V₂|`. Print the results of each iteration.


In [6]:
# --- Problem 1 Solution ---
using LinearAlgebra

# --- Setup System Data ---
# Define Ybus, Scheduled Powers, and Initial Voltages
# ... YOUR CODE HERE ...

z=im*.1
y=1/z
Ybus = [y -y; -y y]
v1 = 1.0 + 0.0im # Corrected initial voltage for bus 1
v1_mag=abs(v1)
delta1=angle(v1)
v2 = 1.0 + 0.0im # Corrected initial voltage for bus 2
v2_mag=abs(v2)
delta2=angle(v2)
tolerance=.001
p2_l=1.8
q2_l=.6


# --- Iteration Loop ---
println("--- Newton-Raphson Iterations ---")
println("Iter | V2_mag  | delta2_deg |   dP2    |   dQ2   ")
println("--------------------------------------------------")

for i in 1:10 # Limit to 10 iterations to prevent infinite loops
    # --- Step 1: Calculate Power Injections (P_calc, Q_calc) ---

    v2_mag=abs(v2)
    delta2=angle(v2)
    v1_mag=abs(v1)
    delta1=angle(v1)

P2_calc = v1_mag*v2_mag*abs(Ybus[2,1])*cos(delta2-delta1-angle(Ybus[2,1])) + v2_mag^2*abs(Ybus[2,2])*cos(-angle(Ybus[2,2]))
Q2_calc = v1_mag*v2_mag*abs(Ybus[2,1])*sin(delta2-delta1-angle(Ybus[2,1])) + v2_mag^2*abs(Ybus[2,2])*sin(-angle(Ybus[2,2]))

    # --- Step 2: Calculate Mismatches and Check for Convergence ---

delta_p2 = -p2_l-P2_calc
delta_q2 = -q2_l-Q2_calc

    # --- Print Iteration Results ---
    println("%4d | %7.4f | %10.4f | %7.4f | %7.4f\n", i, v2_mag, rad2deg(delta2), delta_p2, delta_q2)


    if max(abs(delta_p2), abs(delta_q2)) < tolerance  #From solution file
        println("\nConvergence reached!")
        break
    end

    # --- Step 3: Calculate the 2x2 Jacobian Matrix ---
J = zeros(2, 2)
J[1,1] = -v2_mag*v1_mag*abs(Ybus[2,1])*sin(delta2-delta1-angle(Ybus[2,1])) - 2*v2_mag^2*abs(Ybus[2,2])*sin(-angle(Ybus[2,2])) # dP/ddelta
J[1,2] = v1_mag*abs(Ybus[2,1])*cos(delta2-delta1-angle(Ybus[2,1]))+2*v2_mag*abs(Ybus[2,2])*cos(-angle(Ybus[2,2])) # dP/dV
J[2,1] = v2_mag*v1_mag*abs(Ybus[2,1])*cos(delta2-delta1-angle(Ybus[2,1]))+2*v2_mag^2*abs(Ybus[2,2])*cos(-angle(Ybus[2,2])) # dQ/ddelta
J[2,2] = v1_mag*abs(Ybus[2,1])*sin(delta2-delta1-angle(Ybus[2,1]))+2*v2_mag*abs(Ybus[2,2])*sin(-angle(Ybus[2,2])) # dQ/dV

    # --- Step 4: Solve for Updates ---
    mm=[delta_p2, delta_q2]
    update=inv(J)*mm
    dd2=update[1]
    dv2m=update[2]


    # --- Step 5: Update Voltages ---
nd2=delta2+dd2
nv2m=v2_mag+dv2m
v2=nv2m*exp(im*nd2)

end

--- Newton-Raphson Iterations ---
Iter | V2_mag  | delta2_deg |   dP2    |   dQ2   
--------------------------------------------------
%4d | %7.4f | %10.4f | %7.4f | %7.4f
11.00.0-1.8000000000000014-0.6
%4d | %7.4f | %10.4f | %7.4f | %7.4f
20.9410.313240312354829-3.4828779902027494-0.18786928779165846
%4d | %7.4f | %10.4f | %7.4f | %7.4f
30.845178641606476932.84775150980498-6.38432136686188-0.6427981646661894
%4d | %7.4f | %10.4f | %7.4f | %7.4f
40.4509825009613750466.7031931479491-5.942131869282839-0.8502420039893365
%4d | %7.4f | %10.4f | %7.4f | %7.4f
50.08426886380873733-87.55949264141368-0.9580757016825635-0.635129038200357
%4d | %7.4f | %10.4f | %7.4f | %7.4f
60.17077418127364963-36.921840880805725-0.77411676929361770.4736257557610589
%4d | %7.4f | %10.4f | %7.4f | %7.4f
70.21426812952551455-74.49716837425370.26472464620240355-0.48639960165290846
%4d | %7.4f | %10.4f | %7.4f | %7.4f
80.17932791079867655-62.56311615883533-0.20843080878539633-0.09529360135757536
%4d | %7.4f | %10.4


---
### **Problem 2**

For a three-bus power system, assume bus 1 is the swing bus with a per unit voltage of `1.0∠0°`, bus 2 is a PQ bus with a per unit load of `2.0 + j0.5`, and bus 3 is a PV bus with 1.0 per unit generation and a 1.0 voltage setpoint. The per unit line impedances are `j0.1` between buses 1 and 2, `j0.4` between buses 1 and 3, and `j0.2` between buses 2 and 3. Using a flat start (`1.0∠0°`), use the Newton-Raphson approach to determine the **first iteration** phasor voltages at buses 2 and 3.

**Guidance:**
*   This is the same setup as the NR example from Lecture 11. You are only required to perform **one** full iteration.
*   The unknowns are `δ₂`, `δ₃`, and `|V₂|`. The Jacobian will be 3x3.
*   Follow the same logic as Problem 1: Build Ybus -> Calculate Mismatches -> Calculate 3x3 Jacobian -> Solve for Updates -> Update Voltages.
*   You can reuse and adapt the code from the lecture notebook.

In [ ]:
# --- Problem 2 Solution ---

# --- Setup System Data ---
# Define Ybus, Scheduled Powers, and Initial Voltages
# ... YOUR CODE HERE ...


# --- Calculate Mismatches for Iteration 0 ---
# ... YOUR CODE HERE ...


# --- Calculate the 3x3 Jacobian Matrix for Iteration 0 ---
# ... YOUR CODE HERE ...


# --- Solve for the Update Vector ---
# ... YOUR CODE HERE ...


# --- Calculate the Voltages at the End of Iteration 1 ---
# ... YOUR CODE HERE ...


println("--- Results after 1 Iteration ---")
# Print V2_final and V3_final in polar form
# ... YOUR CODE HERE ...



---
### **Problem 3**

Manually solve the power flow for the following network using the Newton-Raphson approach. Use a 100 MVA base.
*   **Bus 1:** Slack bus, `V = 1.0∠0°` p.u.
*   **Bus 2:** PV bus, `P_G = 80 MW`, `|V| = 1.0` p.u.
*   **Bus 3:** PQ bus, `P_L = 180 MW`, `Q_L = 0` (assumed).
*   Each of the three transmission lines has an impedance of `0.05 + j0.1` p.u.

Iterate until the convergence criteria of 0.1 MVA (0.001 p.u.) is met. Show all your work by printing the results of each iteration. Then, verify your final converged solution by solving the case with `PowerModels.jl`.

**Guidance:**
*   This is a full implementation of the NR loop, similar to Problem 1, but for a 3-bus system.
*   The unknowns are `δ₂`, `δ₃`, and `|V₃|`. The Jacobian will be 3x3.
*   The scheduled powers must be converted to per-unit: `P_G2 = 80/100 = 0.8` p.u., `P_L3 = 180/100 = 1.8` p.u.
*   For verification, you will need to create a `PowerModels.jl` case dictionary that represents this system and then call `solve_ac_pf`.


In [ ]:
# --- Problem 3 Solution ---

# --- Part A: Manual Newton-Raphson Implementation ---

# --- Setup System Data ---
# Define Ybus, Scheduled Powers, and Initial Voltages
# ... YOUR CODE HERE ...


# --- Iteration Loop ---
println("--- Manual Newton-Raphson Iterations ---")
# ... (Use a similar loop structure as Problem 1) ...
# YOUR CODE HERE to implement the full NR loop


# --- Part B: Verification with PowerModels.jl ---

# --- Create the PowerModels Case Dictionary ---
case_p3 = Dict(
    "source_type" => "dict",
    "baseMVA" => 100.0,
    # Define the "bus", "branch", "gen", and "load" dictionaries
    # ... YOUR CODE HERE ...
)

# --- Solve with PowerModels.jl ---
using PowerModels, Ipopt
result_p3 = ... # YOUR CODE HERE: solve_ac_pf(...)

println("\n--- PowerModels.jl Verification ---")
# Print the solution for V2 and V3 from the 'result_p3' dictionary
# ... YOUR CODE HERE ...


---
### **Problem 4: The System Under Stress**

You are a planning engineer. The standard IEEE 14-bus system is experiencing heavy summer loading (all loads are increased by 20%). A critical generator (the one at Bus 2) has also tripped offline.

(a) Run a power flow on this "stressed" 14-bus case and identify any bus voltage magnitudes that are below the emergency limit of **0.94 p.u.**

(b) Propose and implement a **control action** to fix the lowest voltage violation. Your goal is to bring all bus voltages back above 0.94 p.u. A good first step is to add a shunt capacitor bank at the bus with the lowest voltage. You may need to try different sizes for the capacitor.

(c) Run the power flow again on your "fixed" case and present a final report showing that all voltage violations have been resolved.

**Guidance:**
*   This problem uses the skills from the Lecture 12 workshop.
*   You will load the built-in `case14.m` file.
*   Create a `deepcopy` of the case data to create your "stressed" version.
*   To model the generator trip, set the `gen_status` of the generator at bus 2 to `0`.
*   To add a shunt capacitor, you will modify the `bs` value of the bus with the lowest voltage.


In [ ]:
# --- Problem 4 Solution ---
using PowerModels, Ipopt

# --- Step 1 (Part a): Analyze the Stressed Case ---
# Load the case14.m file
case_14bus = ... # YOUR CODE HERE

# Create the "stressed" case by deepcopying, scaling loads, and tripping the generator at bus 2
case_stressed = ... # YOUR CODE HERE

# Solve the power flow for the stressed case
result_stressed = ... # YOUR CODE HERE

println("--- Part (a): Stressed System Voltage Report ---")
# Write a loop to print the voltages and identify any below 0.94 p.u.
# Keep track of the bus with the lowest voltage.
# ... YOUR CODE HERE ...


# --- Step 2 (Part b): Propose and Implement a Fix ---
# Create a new copy of the stressed case for your fix
case_fixed = ... # YOUR CODE HERE

# Add a shunt capacitor (positive 'bs') at the bus you identified
# You may need to experiment with the size of 'bs'. Start with 0.2 p.u.
bus_to_fix = "..." # The bus number with the lowest voltage
capacitor_size_pu = ... # Your chosen size
# ... YOUR CODE HERE to modify the case_fixed dictionary ...


# --- Step 3 (Part c): Verify the Solution ---
# Solve the power flow for your "fixed" case
result_fixed = ... # YOUR CODE HERE

println("\n--- Part (c): Final Corrected System Voltage Report ---")
# Write a final loop to print the new voltages and confirm all are above 0.94 p.u.
# ... YOUR CODE HERE ...